In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

import os 
import tensorflow as tf
from pathlib import Path
import cv2

# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



In [2]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('jupyterlab', 'http://ef139013adca.ngrok.io'), ('tensorboard', 'http://c6fb4f5c0970.ngrok.io')]


In [3]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

uzeros = ['Cardiomegaly',
        'Enlarged Cardiomediastinum',
        'Lung Opacity',
        'Lung Lesion',
        'Consolidation',
        'Pneumothorax',
        'Pleural Effusion']

uones = ['Edema',
        'Atelectasis',
        'Fracture',
        'Pleural Other',
        'Pneumonia',]

upsample_factors = {
    "Enlarged Cardiomediastinum": 1,
    "Lung Lesion":1,
    #"Pneumothorax":1,
    #"Pneumonia":1,
    "Pleural Other":2,
    "Fracture":2,
}
transformations_0 = {"hist_equalization":{}}
transformations_1 = { "gaussian_blur":{"kernal_size":3}, "hist_equalization":{}}
transformations_2 = {"unsharp_mask":{"radius":2, "amount":1}}
transformations_3 = {"windowing"}
transformations_4 = {}

In [6]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                     name="Chexpert_Masked_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled",
                                     classes=columns_12,
                                     #train_labels = "train.csv",
                                     train_labels = "nofinding_train.csv",
                                     test_labels = "test.csv",
                                     nan_replacement = -1, #float("NaN"),
                                     u_enc = [uzeros, uones],
                                     epochs=3,
                                     batch_size=32,
                                     crop = False,
                                     dim=(256, 256),
                                     loss = BinaryCrossentropy(),
                                     use_class_weights = False,
                                     upsample_factors = upsample_factors,
                                     metrics=METRICS,
                                     single_class_metrics=SINGLE_CLASS_METRICS,
                                     optimizer = Adam(learning_rate=1e-4, clipnorm=1),
                                     lr_factor = 0.1,
                                     augmentation = "affine",
                                     transformations = transformations_2,
                                     split_seed = 6122156,
                                     split_valid_size = 0.05, 
                                     preprocess_input_fn = tf.keras.applications.densenet.preprocess_input)


bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
                                                bce_benchmark.negative_weights)

bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, DenseNet169, bce_benchmark.label_columns)

Initialzed custom_binary_crossentropy with epsilon 1e-05


In [7]:
bce_chexpert_exp.benchmark.as_dict()

{'benchmark_name': 'Chexpert_Masked_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled',
 'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 3,
 'optimizer': 'Adam',
 'learning_rate': 0.0001,
 'lr_factor': 0.1,
 'loss': 'weighted_binary_crossentropy',
 'use_class_weights': False,
 'class_weights': [1.0471136569976807,
  0.8615774512290955,
  0.21405130624771118,
  1.2290253639221191,
  0.3829084038734436,
  1.5654457807540894,
  0.9355764389038086,
  0.3477109670639038,
  1.1680023670196533,
  0.27839189767837524,
  1.1709423065185547,
  0.7992541790008545],
 'positive_weights': [1.0635147094726562,
  0.800445020198822,
  0.4360807538032532,
  0.7231208682060242,
  0.5485225915908813,
  1.9478179216384888,
  0.6219006180763245,
  0.44203197956085205,
  1.574169635772705,
  0.6010105609893799,
  0.6640639305114746,
  0.577321469783783],
 'negative_weights': [0.5162432193756104,
  0.61225765943

In [ ]:
bce_chexpert_exp.train()

Epoch 1/3
 825/8344 [=>............................] - ETA: 49:45 - loss: 0.3985 - auc: 0.5994 - precision: 0.2197 - recall: 0.7264 - f2_score: 0.4970 - binary_accuracy: 0.4876 - accuracy_enlarged_cardiomediastinum: 0.4000 - accuracy_cardiomegaly: 0.4993 - accuracy_lung_opacity: 0.6129 - accuracy_lung_lesion: 0.3726 - accuracy_edema: 0.6278 - accuracy_consolidation: 0.5328 - accuracy_pneumonia: 0.4130 - accuracy_atelectasis: 0.4980 - accuracy_pneumothorax: 0.5284 - accuracy_pleural_effusion: 0.6754 - accuracy_pleural_other: 0.3435 - accuracy_fracture: 0.3472 - auc_enlarged_cardiomediastinum: 0.4997 - auc_cardiomegaly: 0.6273 - auc_lung_opacity: 0.6544 - auc_lung_lesion: 0.4874 - auc_edema: 0.7531 - auc_consolidation: 0.6224 - auc_pneumonia: 0.5855 - auc_atelectasis: 0.5985 - auc_pneumothorax: 0.6336 - auc_pleural_effusion: 0.7562 - auc_pleural_other: 0.5227 - auc_fracture: 0.4524 - precision_enlarged_cardiomediastinum: 0.0960 - precision_cardiomegaly: 0.1495 - precision_lung_opacity: 0

In [ ]:
bce_chexpert_exp.evaluate()

In [ ]:
bce_chexpert_exp.save()

'test'